## Import Data

In [11]:
from datasets import load_from_disk
output_dir = r"C:\Users\25811\Desktop\Summer Search\Yu\Data_ds_format"
raw_data = load_from_disk(output_dir)

c:\Microsoft VS Code\VsCodeProject\Prof.Yu\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Each text is a message from users in one turn
labeled_samples = []

for record in raw_data:
    conversation_list = record.get('conversation', [])
    
    conversation_id = record.get('conversation_hash') 
    
    for i, turn in enumerate(conversation_list):
        if 'content' in turn and turn['content']:
            
            content = turn['content']
            role = turn.get('role', 'unknown') 
            
            
            if role == 'user':
            
            # if role in ['user', 'assistant']:
            
                labeled_samples.append({
                    "conversation_hash": conversation_id,
                    "turn_index": i,
                    "role": role,
                    "text": content
                })

print(f"Total samples extracted: {len(labeled_samples)}")

Total samples extracted: 11006


In [ ]:
def normalize_conv(conv):
    if conv is None:
        return []
    if isinstance(conv, dict):
        if "content" in conv or "text" in conv:
            return [conv]
        conv = conv.get("conversation")
        if isinstance(conv, dict):
            return [conv]
    if isinstance(conv, list):
        return conv
    return []

def merge_conversations(raw_data, roles=None, join_with=" ", deduplicate=False, max_conversations=None):
    out = []
    seen = set()
    count = 0
    for record in raw_data:
        if max_conversations and count >= max_conversations:
            break
        conv = normalize_conv(record.get("conversation"))
        if not conv:
            continue
        conversation_id = record.get("conversation_hash")
        turns_texts = []
        turns_meta = []
        for turn in conv:
            if isinstance(turn, str):
                text = turn.strip()
                role = "unknown"
            elif isinstance(turn, dict):
                text = (turn.get("content")).strip()
                role = turn.get("role", "unknown")
            else:
                continue
            if not text:
                continue
            if roles and role not in roles:
                continue
            turns_texts.append(text)
            turns_meta.append({"role": role, "text": text})
        if not turns_texts:
            continue
        merged = join_with.join(turns_texts)
        if deduplicate and merged in seen:
            continue
        seen.add(merged)
        out.append({
            "conversation_hash": conversation_id,
            "merged_text": merged,
            "turns": turns_meta
        })
        count += 1
    return out

merged = merge_conversations(raw_data, roles=["user"], join_with=" ", deduplicate=True, max_conversations=100)
texts_to_label = [item["merged_text"] for item in merged]
print(f"Prepared {len(texts_to_label)} merged texts. Sample 1:\\n", texts_to_label[:1])


Prepared 100 merged texts. Sample 1:\n ['I would liketo be a researcher of artificial intelligence (architecture) and an architectural teacher, requiringyou tostudya given topic, develop thesisstatements,and createconvincing documentsthatareboth informative and innovative, while offering assistance toarchitectsandarchitectswhoneedto improve their writing, and you will beresponsible for providing feedback to architects and architectsusingartificial intelligence toolssuch as natural languageprocessing. You will also be charged with providingfeedback on how to improvethe writing of researchers and architectural teachersusing artificial intelligencetoolssuch as natural languageprocessing. Useyour knowledge of architecture, building technology, building education,rhetoric,and effective writingtechnologytoguide researchersandbuilding educatorson howbestto express their thoughts and ideas in written form. "Write a paragraph in formal&SCI tone, academic writing style  for this title ""A. Syste

In [ ]:
print(labeled_samples[2].get("text"))  




write a  literature review in apa reference stype, aims to comprehensively explore and synthesize available evidence on RTAs and the aftermath trauma for males in Riyadh


In [ ]:
def build_texts_to_label(labeled_samples, roles=None, deduplicate=True, max_samples=None):
    texts = []
    seen = set()
    for item in labeled_samples:
        if not isinstance(item, dict):
            continue
        role = item.get("role", "unknown")
        if roles and role not in roles:
            continue
        txt = (item.get("text") or "").strip()
        if not txt:
            continue
        if deduplicate:
            if txt in seen:
                continue
            seen.add(txt)
        texts.append(txt)
        if max_samples and len(texts) >= max_samples:
            break
    return texts

texts_to_label = build_texts_to_label(labeled_samples, roles=["user"], deduplicate=True, max_samples=10)
print(f"Prepared {len(texts_to_label)} texts (sample 3):", texts_to_label[:3])


Prepared 10 texts (sample 3): ['I would liketo be a researcher of artificial intelligence (architecture) and an architectural teacher, requiringyou tostudya given topic, develop thesisstatements,and createconvincing documentsthatareboth informative and innovative, while offering assistance toarchitectsandarchitectswhoneedto improve their writing, and you will beresponsible for providing feedback to architects and architectsusingartificial intelligence toolssuch as natural languageprocessing. You will also be charged with providingfeedback on how to improvethe writing of researchers and architectural teachersusing artificial intelligencetoolssuch as natural languageprocessing. Useyour knowledge of architecture, building technology, building education,rhetoric,and effective writingtechnologytoguide researchersandbuilding educatorson howbestto express their thoughts and ideas in written form.', '"Write a paragraph in formal&SCI tone, academic writing style  for this title ""A. Systematic 

In [16]:
print(texts_to_label[:5])

['I would liketo be a researcher of artificial intelligence (architecture) and an architectural teacher, requiringyou tostudya given topic, develop thesisstatements,and createconvincing documentsthatareboth informative and innovative, while offering assistance toarchitectsandarchitectswhoneedto improve their writing, and you will beresponsible for providing feedback to architects and architectsusingartificial intelligence toolssuch as natural languageprocessing. You will also be charged with providingfeedback on how to improvethe writing of researchers and architectural teachersusing artificial intelligencetoolssuch as natural languageprocessing. Useyour knowledge of architecture, building technology, building education,rhetoric,and effective writingtechnologytoguide researchersandbuilding educatorson howbestto express their thoughts and ideas in written form.', '"Write a paragraph in formal&SCI tone, academic writing style  for this title ""A. Systematic literature review"" based on t

## Import openAI

In [ ]:
import os
from openai import OpenAI


OPENAI_API_KEY = ""


In [6]:
print(123)
print(OPENAI_API_KEY)

123


In [ ]:


client = OpenAI(
    api_key=OPENAI_API_KEY,
    timeout=60.0  
)

In [ ]:
SYSTEM_PROMPT = """You are an expert data annotator. Your task is to strictly determine whether the content described in the user's text belongs to a "Scientific Activity".

**STRICT OUTPUT RULE: You MUST output ONLY 'Yes' or 'No'. Do not output any explanation, punctuation, or other text.**

## Definition and Boundaries of Scientific Activities

**Definition:** "Scientific activities" refer to any systematic process or behavior within the fields of natural science, social science, or formal science, aimed at acquiring new knowledge, verifying existing theories, or systematically applying scientific methods.

**Mark as 'Yes' (Inclusion):**
1.  Systematic Review and Synthesis: Describes the act of systematically searching, screening, critically appraising, and synthesizing existing research literature to identify gaps, establish the state of the art, or form new hypotheses.
2.  Methodological or Theoretical Critique: Describes the critical analysis of the theories, models, or research methods used in existing literature to verify, challenge, or refine scientific understanding.
3.  Science Communication (Drafting the Review): Describes the act of writing or reporting the results of the literature review process for publication or presentation as part of a research output.

**Mark as 'No' (Exclusion):**
1.  Daily or Business Activities: Describe non-research oriented daily affairs or company/organizational activities mentioned within the text.
2.  Pure education/learning: Describe attending classes, reading textbooks, or performing tasks (including literature summaries/reviews) whose primary purpose is to fulfill a course requirement, demonstrate comprehension, or learn methods, and which do not involve original critical synthesis, new data analysis, or the pursuit of an original research question.
3.  Simple information query: For the inquiry of a certain issue, there is no description of the ongoing systematic research or analysis process
"""

In [ ]:
texts_to_label = [
    "Conducting a meta-analysis of previous studies.",
    "Critically comparing methodological approaches across papers.",
    "Identifying the historical evolution of a theory through literature review.",
    "Analyzing the limitations of a commonly used model in the reviewed papers.",
    "Proposing a theoretical framework based on synthesizing existing concepts.",
    "Writing the Introduction section of a thesis that outlines the research gap.",
    "Drafting the 'Related Work' chapter of a paper.",
    "The library renewed its subscription to the journal.",
    "Please constrcut an abstract of the question A of the exam.",
    "Summarizing the abstract of paper."
]

In [ ]:
def automated_labeling(text_data, system_prompt, model="gpt-4o"):
    """
    使用 OpenAI API 批量标注文本数据。
    """
    labeled_results = []
    
    for text in text_data:
        print(f"Processing text: '{text[:50]}...'")
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Please analyze the following text and provide the 'Yes' or 'No' label: {text}"}
        ]
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.0,
                max_tokens=10
            )
            
            label = response.choices[0].message.content.strip()
            
            labeled_results.append({
                "text": text,
                "label": label
            })
            print(f" -> Labeled as: {label}\n")
            
        except Exception as e:
            print(f"An error occurred during API call for text '{text[:20]}...': {e}")
            labeled_results.append({"text": text, "label": "ERROR"})
            
    return labeled_results



In [59]:
def automated_labeling_with_tracking(text_data, system_prompt, model="gpt-5o-nano"):
    labeled_results = []
    
    total_prompt_tokens = 0
    total_completion_tokens = 0
    
    for text in text_data:
        print(f"Processing text: '{text[:50]}...'")
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Please analyze the following text and provide the 'Yes' or 'No' label: {text}"}
        ]
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages
            )
            
            usage = response.usage
            prompt_tokens = usage.prompt_tokens
            completion_tokens = usage.completion_tokens
            
            total_prompt_tokens += prompt_tokens
            total_completion_tokens += completion_tokens
            
            label = response.choices[0].message.content.strip()
            
            labeled_results.append({
                "text": text,
                "label": label,
                "tokens_used": usage.total_tokens  # token
            })
            
            print(f" -> Labeled as: {label} (Prompt: {prompt_tokens} tokens, Completion: {completion_tokens} tokens)\n")
            
        except Exception as e:
            print(f"An error occurred during API call for text '{text[:20]}...': {e}")
            labeled_results.append({"text": text, "label": "ERROR", "tokens_used": 0})
            
    print(f"\n--- Total Tokens Used Across All Calls ---")
    print(f"Total Prompt Tokens: {total_prompt_tokens}")
    print(f"Total Completion Tokens: {total_completion_tokens}")
    print(f"GRAND TOTAL Tokens: {total_prompt_tokens + total_completion_tokens}")
            
    return labeled_results

# final_labels = automated_labeling_with_tracking(texts_to_label, SYSTEM_PROMPT, model="GPT-5o Nano")

In [ ]:
final_labels = automated_labeling_with_tracking(texts_to_label, SYSTEM_PROMPT,model="gpt-5-nano")

Processing text: 'Conducting a meta-analysis of previous studies....'
 -> Labeled as: Yes (Prompt: 360 tokens, Completion: 74 tokens)

Processing text: 'Critically comparing methodological approaches acr...'
 -> Labeled as: Yes (Prompt: 359 tokens, Completion: 202 tokens)

Processing text: 'Identifying the historical evolution of a theory t...'
 -> Labeled as: Yes (Prompt: 363 tokens, Completion: 202 tokens)

Processing text: 'Analyzing the limitations of a commonly used model...'
 -> Labeled as: Yes (Prompt: 366 tokens, Completion: 266 tokens)

Processing text: 'Proposing a theoretical framework based on synthes...'
 -> Labeled as: Yes (Prompt: 363 tokens, Completion: 522 tokens)

Processing text: 'Writing the Introduction section of a thesis that ...'
 -> Labeled as: Yes (Prompt: 364 tokens, Completion: 906 tokens)

Processing text: 'Drafting the 'Related Work' chapter of a paper....'
 -> Labeled as: Yes (Prompt: 363 tokens, Completion: 266 tokens)

Processing text: 'The library rene

In [45]:
for item in final_labels:
    print(f"Text: {item['text']}")
    print(f"Label: {item['label']}")
    print("-" * 30)

Text: Conducting a meta-analysis of previous studies.
Label: Yes
------------------------------
Text: Critically comparing methodological approaches across papers.
Label: Yes
------------------------------
Text: Identifying the historical evolution of a theory through literature review.
Label: Yes
------------------------------
Text: Analyzing the limitations of a commonly used model in the reviewed papers.
Label: Yes
------------------------------
Text: Proposing a theoretical framework based on synthesizing existing concepts.
Label: Yes
------------------------------
Text: Writing the Introduction section of a thesis that outlines the research gap.
Label: Yes
------------------------------
Text: Drafting the 'Related Work' chapter of a paper.
Label: Yes
------------------------------
Text: The library renewed its subscription to the journal.
Label: No
------------------------------
Text: Please constrcut an abstract of the question A of the exam.
Label: No
---------------------------

In [46]:
texts_to_label_plus = [
    "I am trying to develop a conceptual framework based on synthesizing the principles of operant conditioning and social learning theory.",
    "Which theoretical framework is most popular in the field of cognitive neuroscience literature?",
    "Analyzing the ethical considerations discussed in the literature regarding the use of patient data in machine learning models.",
    "I saw a presentation about meta-analysis yesterday, and now I want to know what the required sample size usually is."
]

In [49]:
final_labels_plus = automated_labeling_with_tracking(texts_to_label_plus, SYSTEM_PROMPT,model="gpt-5-nano")

# for item in final_labels_plus:
#     print(f"Text: {item['text']}")
#     print(f"Label: {item['label']}")
#     print("-" * 30)

Processing text: 'I am trying to develop a conceptual framework base...'
 -> Labeled as: Yes (Prompt: 402 tokens, Completion: 907 tokens)

Processing text: 'Which theoretical framework is most popular in the...'
 -> Labeled as: No (Prompt: 393 tokens, Completion: 458 tokens)

Processing text: 'Analyzing the ethical considerations discussed in ...'
 -> Labeled as: Yes (Prompt: 400 tokens, Completion: 394 tokens)

Processing text: 'I saw a presentation about meta-analysis yesterday...'
 -> Labeled as: No (Prompt: 402 tokens, Completion: 970 tokens)


--- Total Tokens Used Across All Calls ---
Total Prompt Tokens: 1597
Total Completion Tokens: 2729
GRAND TOTAL Tokens: 4326


In [51]:
texts_to_label_2 = [
    "Explain the main arguments of the paper 'A Model for X' in five bullet points and list its authors.",
    "Generating a list of keywords from the top 10 most cited papers to use as input for a future literature search.",
    "alphabetize the bibliography and ensure all citation years are enclosed in parentheses.",
    "Find the full name of the organization abbreviated as 'NIH' from the reference list."
]


In [52]:
final_labels_2 = automated_labeling_with_tracking(texts_to_label_2, SYSTEM_PROMPT,model="gpt-5-nano")


Processing text: 'Explain the main arguments of the paper 'A Model f...'
 -> Labeled as: Yes (Prompt: 401 tokens, Completion: 842 tokens)

Processing text: 'Generating a list of keywords from the top 10 most...'
 -> Labeled as: Yes (Prompt: 403 tokens, Completion: 970 tokens)

Processing text: 'alphabetize the bibliography and ensure all citati...'
 -> Labeled as: No (Prompt: 393 tokens, Completion: 715 tokens)

Processing text: 'Find the full name of the organization abbreviated...'
 -> Labeled as: No (Prompt: 397 tokens, Completion: 714 tokens)


--- Total Tokens Used Across All Calls ---
Total Prompt Tokens: 1594
Total Completion Tokens: 3241
GRAND TOTAL Tokens: 4835


In [64]:
SYSTEM_PROMPT_HW = """You are an expert data annotator. Your task is to strictly determine whether the content described in the user's text belongs to a "Scientific Activity".

**STRICT OUTPUT RULE: You MUST output ONLY 'Yes' or 'No'. Do not output any explanation, punctuation, or other text.**

## Definition and Boundaries of Scientific Activities

**Definition:** "Scientific activities" refer to any systematic process or behavior within the fields of natural science, social science, or formal science, aimed at acquiring new knowledge, verifying existing theories, or systematically applying scientific methods.

**Mark as 'Yes' (Inclusion):**
1.  Systematic Review and Synthesis (except homework requirement): Describes the act of systematically searching, screening, critically appraising, and synthesizing existing research literature to identify gaps, establish the state of the art, or form new hypotheses. However, it should NOT include activities primarily aimed at fulfilling course requirements or learning objectives without original critical synthesis or research effort.
2.  Methodological or Theoretical Critique: Describes the critical analysis of the theories, models, or research methods used in existing literature to verify, challenge, or refine scientific understanding.
3.  Science Communication (Drafting the Review): Describes the act of writing or reporting the results of the literature review process for publication or presentation as part of a research output.

**Mark as 'No' (Exclusion):**
1.  Daily or Business Activities: Describe non-research oriented daily affairs or company/organizational activities mentioned within the text.
2.  Pure education/learning: Describe attending classes, reading textbooks, or performing tasks (including literature summaries/reviews) whose primary purpose is to fulfill a course requirement, demonstrate comprehension, or learn methods, and which do not involve original critical synthesis, new data analysis, or the pursuit of an original research question.
3.  Simple information query: For the inquiry of a certain issue, there is no description of the ongoing systematic research or analysis process
"""

In [65]:

texts_to_label_HW1 = [
    "generating a summary of the first three chapters of the assigned textbook on statistical modeling to prepare for a quiz.",
    "provide a basic definition of 'p-value' and 'confidence interval' as requested on homework sheet.",
    "find an example using the regression analysis for my homework.",
    "write a 600 words literature review based on the question A of the homework"
]

In [66]:
final_labels_HW1 = automated_labeling_with_tracking(texts_to_label_HW1, SYSTEM_PROMPT_HW,model="gpt-5-nano")


Processing text: 'generating a summary of the first three chapters o...'
 -> Labeled as: No (Prompt: 449 tokens, Completion: 394 tokens)

Processing text: 'provide a basic definition of 'p-value' and 'confi...'
 -> Labeled as: No (Prompt: 448 tokens, Completion: 266 tokens)

Processing text: 'find an example using the regression analysis for ...'
 -> Labeled as: No (Prompt: 439 tokens, Completion: 714 tokens)

Processing text: 'write a 600 words literature review based on the q...'
 -> Labeled as: No (Prompt: 443 tokens, Completion: 1291 tokens)


--- Total Tokens Used Across All Calls ---
Total Prompt Tokens: 1779
Total Completion Tokens: 2665
GRAND TOTAL Tokens: 4444


In [67]:
texts_to_label_HW2 = [
    "generating a summary on statistical modeling.",
    "provide a basic definition of 'p-value' and 'confidence interval'.",
    "find an example using the regression analysis."
]

final_labels_HW2 = automated_labeling_with_tracking(texts_to_label_HW2, SYSTEM_PROMPT,model="gpt-5-nano")

Processing text: 'generating a summary on statistical modeling....'
 -> Labeled as: Yes (Prompt: 424 tokens, Completion: 970 tokens)

Processing text: 'provide a basic definition of 'p-value' and 'confi...'
 -> Labeled as: No (Prompt: 431 tokens, Completion: 394 tokens)

Processing text: 'find an example using the regression analysis....'
 -> Labeled as: Yes (Prompt: 425 tokens, Completion: 586 tokens)


--- Total Tokens Used Across All Calls ---
Total Prompt Tokens: 1280
Total Completion Tokens: 1950
GRAND TOTAL Tokens: 3230


## Tag Sample

In [68]:
texts_to_label = build_texts_to_label(labeled_samples, roles=["user"], deduplicate=True, max_samples=10)

import random, json, time, os

def sample_and_label(texts, sample_size=150, batch_size=10, system_prompt=SYSTEM_PROMPT, model="gpt-5-nano", out_path="label_sample_150.json"):
    n = min(sample_size, len(texts))
    sampled = random.sample(texts, n)
    all_results = []

    for i in range(0, n, batch_size):
        batch = sampled[i:i+batch_size]
        print(f"Labeling batch {i//batch_size + 1} / {((n-1)//batch_size)+1} (size={len(batch)})")
        try:
            batch_results = automated_labeling_with_tracking(batch, system_prompt, model=model)
            all_results.extend(batch_results)
        except Exception as e:
            print(f"Batch error: {e}")
            for t in batch:
                all_results.append({"text": t, "label": "ERROR", "tokens_used": 0})
        time.sleep(1.5)

        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"Done. Saved {len(all_results)} labeled items to {os.path.abspath(out_path)}")
    return all_results

sampled_labels = sample_and_label(texts_to_label, sample_size=150, batch_size=10, model="gpt-5-nano", out_path="label_sample_150.json")


Labeling batch 1 / 1 (size=10)
Processing text: '"Write a paragraph in formal&SCI tone, academic wr...'
 -> Labeled as: Yes (Prompt: 675 tokens, Completion: 394 tokens)

Processing text: '"Write a paragraph in formal&SCI tone, academic wr...'
 -> Labeled as: Yes (Prompt: 664 tokens, Completion: 202 tokens)

Processing text: 'I would liketo be a researcher of artificial intel...'
 -> Labeled as: Yes (Prompt: 574 tokens, Completion: 842 tokens)

Processing text: '"Write a paragraph in formal&SCI tone, academic wr...'
 -> Labeled as: Yes (Prompt: 675 tokens, Completion: 138 tokens)

Processing text: '"Write a paragraph in formal&SCI tone, academic wr...'
 -> Labeled as: Yes (Prompt: 670 tokens, Completion: 330 tokens)

Processing text: 'write a  literature review in apa reference stype,...'


KeyboardInterrupt: 

In [ ]:
texts_to_label = build_texts_to_label(labeled_samples, roles=["user"], deduplicate=True, max_samples=15000)

In [ ]:
import random, time

def sample_and_label_in_memory(texts, sample_size=1000, batch_size=10, system_prompt=SYSTEM_PROMPT, model="gpt-5-nano"):
    """
    Randomly sample from texts and batch call the annotations, 
    and save the results in an in-memory variable (do not write to a file).
    Return sampled_labels: list of dict {text, prompt, label, tokens_used}
    """

    n = min(sample_size, len(texts))
    sampled = random.sample(texts, n)
    sampled_labels = []

    for i in range(0, n, batch_size):
        batch = sampled[i:i+batch_size]
        print(f"Labeling batch {i//batch_size + 1} / {((n-1)//batch_size)+1} (size={len(batch)})")
        try:
            batch_results = automated_labeling_with_tracking(batch, system_prompt, model=model)
            for text, res in zip(batch, batch_results):
                user_prompt = f"Please analyze the following text and provide the 'Yes' or 'No' label: {text}"
                sampled_labels.append({
                    "text": text,
                    "prompt": {"system": system_prompt, "user": user_prompt},
                    "label": res.get("label", "ERROR"),
                    "tokens_used": res.get("tokens_used", 0)
                })
        except Exception as e:
            print(f"Batch error: {e}")
            for text in batch:
                user_prompt = f"Please analyze the following text and provide the 'Yes' or 'No' label: {text}"
                sampled_labels.append({
                    "text": text,
                    "prompt": {"system": system_prompt, "user": user_prompt},
                    "label": "ERROR",
                    "tokens_used": 0
                })
        time.sleep(1.5)

    print(f"Done. Collected {len(sampled_labels)} labeled items in memory.")
    return sampled_labels

sampled_labels_in_memory = sample_and_label_in_memory(texts_to_label, sample_size=750, batch_size=10, model="gpt-5-nano")

Labeling batch 1 / 75 (size=10)
Processing text: 'Can you send me an URL where i can find the articl...'
 -> Labeled as: No (Prompt: 473 tokens, Completion: 586 tokens)

Processing text: 'make past tense; The composition of participants i...'
 -> Labeled as: Yes (Prompt: 591 tokens, Completion: 330 tokens)

Processing text: 'Are there any relevant reference cases to learn fr...'
 -> Labeled as: No (Prompt: 427 tokens, Completion: 714 tokens)

Processing text: 'I want in same format i gave to you...'
 -> Labeled as: No (Prompt: 426 tokens, Completion: 202 tokens)

Processing text: 'Put this paper in (AEA) style.

Austin Belman 
ECO...'
 -> Labeled as: Yes (Prompt: 1357 tokens, Completion: 522 tokens)

Processing text: 'Ok, here is the first information chunk..

CHUNK
-...'
 -> Labeled as: No (Prompt: 1738 tokens, Completion: 266 tokens)

Processing text: '4. RESULTS
4.1. Bivariate analysis
First, we want ...'
 -> Labeled as: Yes (Prompt: 2391 tokens, Completion: 202 tokens)

Processing 

In [83]:
import pandas as pd, os

df = pd.DataFrame(sampled_labels_in_memory)


df_text_label = df[['text', 'label']].copy()

out_csv = "sampled_labels_text_label750.csv"
df_text_label.to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"Saved {out_csv} ({len(df_text_label)} rows)")
df_text_label.head()


Saved sampled_labels_text_label750.csv (750 rows)


,text,label
0,Can you send me an URL where i can find the ar...,No
1,make past tense; The composition of participan...,Yes
2,Are there any relevant reference cases to lear...,No
3,I want in same format i gave to you,No
4,Put this paper in (AEA) style.\n\nAustin Belma...,Yes


In [ ]:
import pandas as pd

files = [
    "sampled_labels_text_label750.csv",
    "sampled_labels_text_label300.csv",
    "sampled_labels_text_label.csv",
    "sampled_labels_text_Y.csv",
]

dfs = []
for f in files:
    df_tmp = pd.read_csv(
        f,
        encoding="utf-8-sig",
        on_bad_lines="skip"
    )
    print(f"{f}: kept {len(df_tmp)} rows")
    dfs.append(df_tmp)

merged_df = pd.concat(dfs, ignore_index=True)

out_csv_merged = "sampled_labels_text_label_merged.csv"
merged_df.to_csv(out_csv_merged, index=False, encoding="utf-8-sig")

print(f"Saved merged file: {out_csv_merged} (rows={len(merged_df)})")
merged_df.head()

sampled_labels_text_label750.csv: kept 750 rows
sampled_labels_text_label300.csv: kept 300 rows
sampled_labels_text_label.csv: kept 143 rows
sampled_labels_text_Y.csv: kept 148 rows
Saved merged file: sampled_labels_text_label_merged.csv (rows=1341)


,text,label
0,Can you send me an URL where i can find the ar...,No
1,make past tense; The composition of participan...,Yes
2,Are there any relevant reference cases to lear...,No
3,I want in same format i gave to you,No
4,Put this paper in (AEA) style.\n\nAustin Belma...,Yes


Unique labels (raw): ['No' 'Yes']
Unique labels (stripped): ['No' 'Yes']
Total rows: 1341
Rows with label in {Yes, No}: 1341
Rows with other labels: 0
Non-Yes/No labels and counts:
Series([], Name: count, dtype: int64)
